In [1]:
import numpy as np
import requests
import time

from keras.optimizers import *
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors

from distkeras.trainers import *
from distkeras.predictors import *
from distkeras.transformers import *
from distkeras.evaluators import *
from distkeras.utils import *

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
# Load the dataset and labels
x=np.load('RMCx.npy')
y=np.load('RMCy.npy')

In [3]:
conf = SparkConf()
sc = SparkContext(conf = conf)
spark = SQLContext(sc)

In [4]:
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=np.unique(y).shape[0], n_init=10)
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
    r=metrics.homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    R.append(r)

In [5]:
#Arrange feature accroding to thier ranks
Rnk=np.argsort(np.array(R))

In [6]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [7]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    lpa=np.zeros((0,2))
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"])
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"])
        model = Sequential()
        model.add(Dense(128,input_dim=fd.shape[1],activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(64,activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(32,activation='relu',use_bias=True))
        model.add(Dropout(0.5))
        model.add(Dense(np.unique(y).shape[0],activation='softmax',use_bias=True)) #The number of neurons is equal to the number of classes
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
        trainer = SingleTrainer(keras_model=model, worker_optimizer='adam', loss='categorical_crossentropy', num_epoch=1000)
        trained_model = trainer.train(TrD)
        st = time.time()
        predictor = ModelPredictor(keras_model=trained_model)
        et=time.time()-st
        ff=predictor.predict(TsD)
        ts=np.array(map(lambda x: x[0],ff.select('prediction').collect())).reshape(-1,1)
        pp=pp+metrics.accuracy_score(y[test].reshape(-1,1),(ts>=0.5).astype(int))
        lpa=np.vstack((lpa,np.hstack((y[test].reshape(-1,1),ts))))
    pp=pp/kfolds.n_splits
    np.savetxt('F%d.csv'%j,lpa,delimiter=',')
    smr.append([j, pp, et*1000000/x.shape[0]]) #Calculate the time required to predict a label per each object in uS.

In [8]:
smr

[[78.0, 0.9955, 0.6404],
 [77.0, 0.9955, 0.6405],
 [76.0, 0.9954, 0.6623],
 [75.0, 0.9957, 0.703],
 [74.0, 0.9946, 0.7162],
 [73.0, 0.9952, 0.7233],
 [72.0, 0.9956, 0.7358],
 [71.0, 0.9955, 0.7991],
 [70.0, 0.995, 0.7815],
 [69.0, 0.9949, 0.7967],
 [68.0, 0.9951, 0.8449],
 [67.0, 0.9956, 0.8591],
 [66.0, 0.9956, 0.8748],
 [65.0, 0.9955, 0.903],
 [64.0, 0.9954, 0.9242],
 [63.0, 0.9953, 0.9478],
 [62.0, 0.9946, 0.9436],
 [61.0, 0.9942, 0.9775],
 [60.0, 0.9936, 0.9953],
 [59.0, 0.9921, 1.0157],
 [58.0, 0.9933, 1.0408],
 [57.0, 0.9913, 1.0403],
 [56.0, 0.9922, 1.0757],
 [55.0, 0.9927, 1.0942],
 [54.0, 0.993, 1.1314],
 [53.0, 0.9912, 1.1413],
 [52.0, 0.9911, 1.1775],
 [51.0, 0.992, 1.2052],
 [50.0, 0.9906, 1.2122],
 [49.0, 0.992, 1.2313],
 [48.0, 0.9908, 1.2612],
 [47.0, 0.9911, 1.2686],
 [46.0, 0.9896, 1.3062],
 [45.0, 0.9903, 1.3301],
 [44.0, 0.9904, 1.3485],
 [43.0, 0.991, 1.3747],
 [42.0, 0.9901, 1.3918],
 [41.0, 0.9899, 1.423],
 [40.0, 0.9905, 1.4464],
 [39.0, 0.9908, 1.704],
 [38.0, 0